# Intro to Google Earth Engine Python API

Based on GEE intro Python API tutorial [here](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard#exploration_of_the_earth_engine_data_catalog)

---

### Download and Install Python API

Follow instructions in `install_GEE.ipynb`

In [1]:
# Import modules
import ee
import folium
from branca.element import Figure


Before we can get started, we need to initialize Google Earth Engine. This **MUST** be done every time we want to use GEE in python.

In [2]:
# Initialize the Google Earth Engine module
ee.Initialize()

In [3]:
# Print metadata for a DEM dataset
print(ee.Image('USGS/SRTMGL1_003').getInfo())

{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [1296001, 417601], 'crs': 'EPSG:4326', 'crs_transform': [0.0002777777777777778, 0, -180.0001388888889, 0, -0.0002777777777777778, 60.00013888888889]}], 'id': 'USGS/SRTMGL1_003', 'version': 1641990767055141, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'Image', 'keywords': ['dem', 'elevation', 'geophysical', 'nasa', 'srtm', 'topography', 'usgs'], 'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png', 'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JPL\nat a resolution of 1 arc-second (approximately 30m).</p><p>This dataset

In [4]:
# from https://spatial.utk.edu/maps/ee-api-folium-setup.html
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

## Dataset Types

<li><i>Features</i> are like vectors/shapefiles. They are a geometric objects with a list of properties (attribute table).</li>

<li><i>Images</i> are like rasters. They can include several bands.</li>

<li><i>Collections</i> are groups of Features or Images. A shapefile of US states would be an `ee.FeatureCollection` and the Landsat dataset would be an `ee.ImageCollection`.</li>

## Mapping with Features

### 1. Make a map

In [5]:
# make a map with center set on Raleigh

# set figure hight and width
fig = Figure(width=600, height=400)

# set the center lat and lon of the map
lat, lon = 35.779, -78.638

# create the map centered on the lat and lon above, set the initial zoom level
raleigh_map = folium.Map(location=[lat, lon], zoom_start=5)
# display the map
fig.add_child(raleigh_map)

### 2. Import state boundaries

In [8]:
# import FeatureCollection of US states from TIGER: US Census States 2018
states = ee.FeatureCollection("TIGER/2018/States")

# Set visualization parameters.
vis_params = {'palette': ['000000']}

# add states to map
raleigh_map.add_ee_layer(states, vis_params, 'states')
# display the map
fig.add_child(raleigh_map)

### 3. Subset to a state

In [7]:
# subset to North Carolina
nc = states.filter(ee.Filter.eq('STATEFP','37'))

# add the North Carolina polygon in red to the map
raleigh_map.add_ee_layer(nc, {'palette':'red'},'NC')
# display the map
fig.add_child(raleigh_map)

## Mapping with Images

### 1. Import data (NLCD)

In [16]:
# make a new map
fig2 = Figure(width=600, height=400)

lat, lon = 35.779, -78.638

raleigh_map2 = folium.Map(location=[lat, lon], zoom_start=10)
fig2.add_child(raleigh_map2)

In [17]:
# import NLCD
nlcd = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/NLCD')

# subset to only 2016 product and select land cover band
nlcd16 = nlcd.filter(ee.Filter.eq('system:index','2021')).first().select('landcover')

# nlcd has a built-in color palette, so we can add the layer with a blank palette input
raleigh_map2.add_ee_layer(nlcd16, {}, 'NLCD Land Cover')

# add layer control panel to map
raleigh_map2.add_child(folium.LayerControl())
fig2.add_child(raleigh_map2)


## Clip Image by Feature

In [18]:
# import county layer subset to North Carolina
nc_counties = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq('STATEFP', '37'))

# create a map
fig3 = Figure(width=600, height=400)

raleigh_map3 = folium.Map(location=[lat, lon], zoom_start=10)

raleigh_map3.add_ee_layer(nc_counties, {}, 'counties')
fig3.add_child(raleigh_map3)

In [19]:
# filter to Wake county
wake = nc_counties.filter(ee.Filter.eq('NAME','Wake'))

# set the color of the polygon for Wake county to yellow
raleigh_map3.add_ee_layer(wake, {'palette':'#e31a1c'}, 'wake')
fig3.add_child(raleigh_map3)

In [20]:
# clip image by Wake county

nlcd_wake = nlcd16.clip(wake)

# add the clip to the map
raleigh_map3.add_ee_layer(nlcd_wake, {}, 'nlcd')
fig3.add_child(raleigh_map3)


## Additional Resources

[GeoPython 2021](https://geemap.org/workshops/GeoPython_2021/) has more extensive examples of using the GEE Python API with an additional python package to build interactive maps that are more like the javascript maps.

[GEE Intro Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard) is the tutorial on which this notebook is based.

[GEE GitHub](https://github.com/google/earthengine-community/tree/master/guides/linked) has many python notebooks that you can explore to learn more about GEE utilitiy.